In [1]:
import pandas as pd

# Rutas de los archivos CSV
ruta_hora = '/content/HORA.csv'
ruta_dia = '/content/DÍA.csv'
ruta_edad_sexo = '/content/EDAD Y SEXO.csv'
ruta_distrito = '/content/DISTRITO.csv'
df_distrito = pd.read_csv(ruta_distrito)
ruta_tipo_accidente = '/content/TIPO DE ACCIDENTE.csv'

**DISTRITO**

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

df_distrito = pd.read_csv(ruta_distrito)
columns_to_scale = ['Accidentes', 'Heridos', 'Muertos']   #Nos centramos en las columnas numéricas
df_distrito_numeric = df_distrito[columns_to_scale]

scaler = StandardScaler() #Escalamos los datos
df_distrito_scaled = scaler.fit_transform(df_distrito_numeric)

sum_of_squared_distances = []
K = range(1, 15)

# Calculamos la suma de las distancias al cuadrado para diferentes valores de k
for k in K:
    km = KMeans(n_clusters=k, random_state=42)
    km = km.fit(df_distrito_scaled)
    sum_of_squared_distances.append(km.inertia_)


plt.figure(figsize=(10, 6))
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('Número de Clústeres')
plt.ylabel('Suma de las distancias al cuadrado')
plt.title('Método del Codo para determinar el K óptimo')
plt.show()

In [ ]:
# Agrupamos los datos de 'Distrito' sumando los totales de 'Accidentes', 'Heridos' y 'Muertos'
df_distrito_aggregated = df_distrito.groupby('Distrito').sum().drop(columns='Año')

# Normalizamos los datos mencionados arriba.
df_distrito_aggregated_scaled = scaler.fit_transform(df_distrito_aggregated)

# Aplicamos K-Means con 4 clústeres en los datos gracias al método del CODO
kmeans_aggregated = KMeans(n_clusters=4, random_state=42)
kmeans_aggregated.fit(df_distrito_aggregated_scaled)

# Asignamos los clústeres optimizados
df_distrito_aggregated['Optimal_Cluster'] = kmeans_aggregated.labels_

# Calculamos las medias de cada clúster para los datos
cluster_means_aggregated = df_distrito_aggregated.groupby('Optimal_Cluster').mean()

cluster_means_aggregated

**DÍA**

In [ ]:
import pandas as pd

file_path = '/content/DÍA.csv'
data = pd.read_csv(file_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

data_grouped = data.groupby('Dia').mean().reset_index()

days_order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_grouped['Dia'] = pd.Categorical(data_grouped['Dia'], categories=days_order, ordered=True)
data_grouped = data_grouped.sort_values('Dia')

# Prepararamos los datos para el clustering, excluyendo columnas NO numéricas
data_clustering_avg = data_grouped.drop('Dia', axis=1)

# Clustering jerárquico
Z_avg = linkage(data_clustering_avg, method='ward')

# Creamos y Mostramos el dendrograma
plt.figure(figsize=(10, 7))
dendrogram(Z_avg, labels=data_grouped['Dia'].values)
plt.title('Dendrograma del Clustering Jerárquico con sus medias')
plt.xlabel('Día')
plt.ylabel('Distancia')
plt.show()


In [ ]:
# Determinamos el número óptimo de clusters en el dendrograma
from scipy.cluster.hierarchy import inconsistent
from scipy.cluster.hierarchy import fcluster

# Profundidad para el cálculo de inconsistencia
depth = 2
incons = inconsistent(Z_avg, depth)
incons[-10:]  # Mostramos las últimas 10 inconsistencias para evaluar

k = 2
clusters = fcluster(Z_avg, k, criterion='maxclust')
data_grouped['Cluster'] = clusters
data_grouped[['Dia', 'Cluster']].sort_values('Cluster')

In [ ]:
# Calculamos la media de accidentes, heridos y muertos para cada cluster
cluster_averages = data_grouped.groupby('Cluster')['Accidentes', 'Heridos', 'Muertos'].mean()
cluster_averages

In [ ]:
# Aquí identificamos el día con el mayor número de accidentes, heridos y muertos dentro de cada cluster
max_values_per_cluster = data_grouped.groupby('Cluster').apply(lambda x: x.nlargest(1, ['Accidentes', 'Heridos', 'Muertos']))
max_values_per_cluster[['Dia', 'Accidentes', 'Heridos', 'Muertos']].reset_index(drop=True)

In [ ]:
import seaborn as sns
# Como antes, creamos la variable Severidad combinando los datos de DÍA (sumando heridos y muertos)
dia_data['Severidad'] = dia_data['Heridos'] + dia_data['Muertos']

# Visualizamos la severidad de los accidentes por día de la semana
plt.figure(figsize=(10, 6))
sns.barplot(x='Dia', y='Severidad', data=dia_data, palette='coolwarm', order=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'])
plt.title('GRAVEDAD de los Accidentes por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Severidad (Heridos + Muertos)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

**HORA**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

data = pd.read_csv('/content/HORA.csv')

# Arupamos los datos de la columna 'Hora' calculando la suma de 'Accidentes', 'Heridos', y 'Muertos'
grouped_data = data.groupby('Hora').agg({'Accidentes': 'sum', 'Heridos': 'sum', 'Muertos': 'sum'}).reset_index()

# Prepararamos los datos para el clustering
X = grouped_data[['Accidentes', 'Heridos', 'Muertos']]

# Escalamos los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Clustering jerárquico
Z = linkage(X_scaled, method='ward')

# Dibujamos el dendrograma
plt.figure(figsize=(10, 8))
dendrogram(Z, labels=grouped_data['Hora'].values, leaf_rotation=90, leaf_font_size=10)
plt.title("Dendrograma del Clustering Jerárquico")
plt.xlabel("Hora")
plt.ylabel("Distancia")
plt.tight_layout()
plt.show()

In [ ]:
from scipy.cluster.hierarchy import fcluster


k = 4  # VIENDO EL DENDOGRAMA, CONSIDERAMOS QUE HAY 4

# Asignamos los clusters a cada observación
clusters = fcluster(Z, k, criterion='maxclust')

grouped_data['Cluster'] = clusters

# Calculamos la media de accidentes, heridos y muertos por cluster
cluster_means = grouped_data.groupby('Cluster').agg({'Accidentes': 'mean', 'Heridos': 'mean', 'Muertos': 'mean'}).reset_index()

cluster_means

In [ ]:
# Creamos un resumen que muestre las horas que pertenecen a cada cluster
cluster_hours = grouped_data.groupby('Cluster')['Hora'].apply(list).reset_index()

cluster_hours

In [ ]:
import matplotlib.pyplot as plt


cluster_hours = pd.DataFrame({
    'Cluster': [1, 2, 3, 4],
    'Horas': [
        ['De 00 a 01', 'De 01 a 02', 'De 02 a 03', 'De 03 a 04', 'De 04 a 05', 'De 05 a 06', 'De 06 a 07'],
        ['De 14 a 15', 'De 20 a 21', 'De 21 a 22'],
        ['De 07 a 08', 'De 08 a 09', 'De 10 a 11', 'De 22 a 23', 'De 23 a 24'],
        ['De 09 a 10', 'De 11 a 12', 'De 12 a 13', 'De 13 a 14', 'De 15 a 16', 'De 16 a 17', 'De 17 a 18', 'De 18 a 19', 'De 19 a 20']
    ]
})

import numpy as np


cluster_data = {
    'Cluster 1': ['De 00 a 01', 'De 01 a 02', 'De 02 a 03', 'De 03 a 04', 'De 04 a 05', 'De 05 a 06', 'De 06 a 07'],
    'Cluster 2': ['De 14 a 15', 'De 20 a 21', 'De 21 a 22'],
    'Cluster 3': ['De 07 a 08', 'De 08 a 09', 'De 10 a 11', 'De 22 a 23', 'De 23 a 24'],
    'Cluster 4': ['De 09 a 10', 'De 11 a 12', 'De 12 a 13', 'De 13 a 14', 'De 15 a 16', 'De 16 a 17', 'De 17 a 18', 'De 18 a 19', 'De 19 a 20']
}

# Convertimos los rangos horarios a valores numéricos para el eje X (solo la hora de inicio)
num_hours = {time_range: int(time_range.split(' ')[1].split('a')[0]) for cluster in cluster_data.values() for time_range in cluster}

# Prepararamos los datos para el gráfico
x_vals = []
y_vals = []
sizes = []
labels = []

for cluster, hours in cluster_data.items():
    for hour in hours:
        x_vals.append(num_hours[hour])
        y_vals.append(cluster)
        sizes.append(300)  # tamaño constante para todos los puntos a 300

# Creamos el gráfico de dispersión
plt.figure(figsize=(14, 6))
scatter = plt.scatter(x_vals, y_vals, s=sizes, c=np.arange(len(x_vals)), cmap="viridis", alpha=0.6, edgecolors="w", linewidth=0.5)

# Añadimos las etiquetas de hora a los puntos
for i, label in enumerate(labels):
    plt.annotate(label, (x_vals[i], y_vals[i]), textcoords="offset points", xytext=(0,10), ha='center')

plt.title('Distribución Horaria por Cluster')
plt.xlabel('Hora del Día')
plt.ylabel('Cluster')
plt.yticks([cluster for cluster in cluster_data.keys()])
plt.xticks(range(0, 24), [f'{i}:00' for i in range(0, 24)], rotation=90)  # cada hora del día
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

**EDAD Y SEXO**

In [3]:
import pandas as pd

archivo = '/content/EDAD Y SEXO.csv'
data = pd.read_csv(archivo)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 8))

sns.barplot(data=data, x='Rango Edad', y='Accidentes Ambos Sexos', color='blue', label='Accidentes')
sns.barplot(data=data, x='Rango Edad', y='Heridos Ambos Sexos', color='orange', label='Heridos')
sns.barplot(data=data, x='Rango Edad', y='Muertos Ambos Sexos', color='red', label='Muertos')

plt.title('Accidentes, Heridos y Muertos por Rango de Edad')
plt.xticks(rotation=45)
plt.ylabel('Cantidad')
plt.xlabel('Rango de Edad')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(16, 10))

# Accidentes a lo largo del tiempo por rango de edad
sns.lineplot(data=data, x='Año', y='Accidentes Ambos Sexos', hue='Rango Edad', marker='o', palette='tab10', linewidth=2.5)

plt.title('Evolución de los Accidentes por Rango de Edad a lo Largo del Tiempo')
plt.xlabel('Año')
plt.ylabel('Número de Accidentes')
plt.legend(title='Rango de Edad', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

**TIPO DE ACCIDENTE**

In [ ]:
import pandas as pd

data_path = '/content/TIPO DE ACCIDENTE.csv'
data = pd.read_csv(data_path)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Seleccionamos solo las columnas numéricas para el clustering
data_clustering = data[['Accidentes', 'Heridos', 'Muertos']]

# Normalizamos las variables
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_clustering)

# Método del codo para determinar el número óptimo de clusters
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_normalized)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.title('Método del Codo para Determinar el Número Óptimo de Clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('SSE (Suma de Cuadrados de las Distancias)')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()


In [ ]:
# Agrupamos por 'Tipo de Accidente' y calculamos la media de las variables numéricas
data_aggregated = data.groupby('Tipo de Accidente').agg({'Accidentes': 'mean', 'Heridos': 'mean', 'Muertos': 'mean'}).reset_index()

# Normalizamos los datos agrupados
data_aggregated_normalized = scaler.fit_transform(data_aggregated[['Accidentes', 'Heridos', 'Muertos']])

# Aplicamos KMeans con 3 clusters a los datos agrupados y normalizados
kmeans_aggregated = KMeans(n_clusters=3, random_state=42)
data_aggregated['Cluster'] = kmeans_aggregated.fit_predict(data_aggregated_normalized)

clusters_summary_aggregated = data_aggregated.groupby('Cluster').agg(
    {'Accidentes': ['mean', 'min', 'max'],
     'Heridos': ['mean', 'min', 'max'],
     'Muertos': ['mean', 'min', 'max'],
     'Tipo de Accidente': 'count'}
).rename(columns={'Tipo de Accidente': 'Cantidad de Tipos de Accidente'})

clusters_summary_aggregated.round(2)

In [ ]:
# Obtenemos los tipos de accidentes en cada cluster
tipos_de_accidente_por_cluster = data_aggregated.groupby('Cluster')['Tipo de Accidente'].apply(list).reset_index()

tipos_de_accidente_por_cluster

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

# Reducción de dimensionalidad para la visualización
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_aggregated_normalized)

# Agregamos las componentes principales al dataframe
data_aggregated['PC1'] = data_pca[:, 0]
data_aggregated['PC2'] = data_pca[:, 1]

# Gráfico de dispersión de los clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(data=data_aggregated, x='PC1', y='PC2', hue='Cluster', palette='viridis', s=100, alpha=0.7, legend="full")

for i, txt in enumerate(data_aggregated['Tipo de Accidente']):
    plt.annotate(txt, (data_aggregated['PC1'][i], data_aggregated['PC2'][i]), textcoords="offset points", xytext=(0,10), ha='center', size=8)

plt.title('Visualización de Clusters de Tipos de Accidente con PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()
